In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.ss.com/lv/real-estate/wood/"
baseurl = "https://www.ss.com"
postfix = "sell/"

def getUrls(url):
    req = requests.get(url)
    if req.status_code != 200:
        print("Status Code is ", req.status_code, "exiting")
        return []

    soup = BeautifulSoup(req.text, 'lxml')
    anchorlist = soup.find_all('a', {'class': 'a_category'})

    finalurls = []
    for elem in anchorlist:
        finalurls.append(baseurl + elem['href'] + postfix)
    len(finalurls)
    return finalurls



In [2]:
workurls = getUrls(url)
workurls[0]

'https://www.ss.com/lv/real-estate/wood/riga-region/sell/'

In [3]:
rurl = workurls[0]

In [4]:
def getAds(url):
    req = requests.get(url)
    
    if req.status_code != 200:
        print("Not a good status code:", req.status_code, "exiting")
        return
    
    rsoup = BeautifulSoup(req.text, 'lxml')
    allrows = rsoup.find_all('tr')
    
    cleanrows = []
    for row in allrows:
        if 'id' in row.attrs.keys() and 'tr_' in row['id']:
            cleanrows.append(row)
    print(len(cleanrows))
    cleanrows = cleanrows[:-1] #alternative to .pop
    # cleanrows.pop() # we do not need the last row
    print(len(cleanrows))
    return cleanrows

In [6]:
clrows = getAds(rurl)
clrows[0]

12
11


<tr id="tr_45836128"><td class="msga2 pp0"><input id="c45836128" name="mid[]" type="checkbox" value="45836128_24629_0"/></td><td class="msga2"><a href="/msg/lv/real-estate/wood/riga-region/stopinu-nov/gcdeb.html" id="im45836128"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/3/517/129155/25830887.th2.jpg"/></a></td><td class="msg2"><div class="d1"><a class="am" data="JTk5JUEyJTk2JTkyaGUlQTAlRUMlOTMlOEViZyU5QSVBMyU5NyU4RGJoJTlFJUE4JTkyJTkxYmg=|hpbY14" href="/msg/lv/real-estate/wood/riga-region/stopinu-nov/gcdeb.html" id="dm_45836128"><b>Pārdod cirsmu 80m3 Priedes.</b></a></div></td><td c="1" class="msga2-o pp6" nowrap=""><b>Stopiņu nov.<br/>Upeslejas</b></td><td c="1" class="msga2-o pp6" nowrap=""><b>80 m²</b></td><td c="1" class="msga2-o pp6" nowrap=""><b>5,000</b>  €</td></tr>

In [10]:
def parseRow(row, base = 'https://www.ss.com'):
    fcells = row.find_all('td')
    if len(fcells) !=6:
        print("Unexpected number of data cells", len(fcells))
        return None
    
    mydict = {}
    mydict['description'] = fcells[2].text
    mydict['region'] = fcells[3].text
    mydict['size'] = fcells[4].text
    # we only need the first part of price as all prices are in Euros, assumption here
    mydict['price'] = fcells[5].text.split()[0]
    mydict['url'] = base + fcells[1].find('a')['href']
    
    return mydict

In [11]:
testad = parseRow(clrows[0])
testad

{'description': 'Pārdod cirsmu 80m3 Priedes.',
 'region': 'Stopiņu nov.Upeslejas',
 'size': '80 m²',
 'price': '5,000',
 'url': 'https://www.ss.com/msg/lv/real-estate/wood/riga-region/stopinu-nov/gcdeb.html'}

In [12]:
def getRows(rowlist):
    results = []
    for row in rowlist:
        results.append(parseRow(row))
    return results

In [13]:
import json
def saveResults(filename, results):
    with open(filename, "w", encoding = 'UTF-8') as write_file:
        json.dump(results, write_file, indent = 4)
        

In [15]:
myresults = getRows(clrows)
myresults

[{'description': 'Pārdod cirsmu 80m3 Priedes.',
  'region': 'Stopiņu nov.Upeslejas',
  'size': '80 m²',
  'price': '5,000',
  'url': 'https://www.ss.com/msg/lv/real-estate/wood/riga-region/stopinu-nov/gcdeb.html'},
 {'description': 'Īpašnieks pārdod mežu Ķekavas nov. z/s "Veckausiņi" 7.4 ha platībā (ozols, priede, e',
  'region': 'Ķekavas pag.Ķekava',
  'size': '7.40 ha.',
  'price': '80,000',
  'url': 'https://www.ss.com/msg/lv/real-estate/wood/riga-region/kekavas-pag/efomp.html'},
 {'description': 'Pārdodu īpašumu',
  'region': 'Stopiņu nov.Saurieši',
  'size': '11.40 ha.',
  'price': '20,000',
  'url': 'https://www.ss.com/msg/lv/real-estate/wood/riga-region/stopinu-nov/hfdim.html'},
 {'description': 'Īpašnieks pārdod cirsmas. Ir ciršanas apliecinājums. Galvenās cirtes. Dastotas. 0, 2',
  'region': 'Krimuldas pag.Sunīši',
  'size': '4.53 ha.',
  'price': '10,500',
  'url': 'https://www.ss.com/msg/lv/real-estate/wood/riga-region/krimuldas-pag/blcjk.html'},
 {'description': 'AS Privatb

In [16]:
saveResults('myforest.json', myresults)